# Cathode test

This notebook can be used to quickly visualize ZDF files.

## Simulation setup

```c++
    uint2 ntiles = {16, 16};
    uint2 nx = {16,16};

    float2 box = {25.6, 25.6};
    float dt = 0.07;

    Simulation sim( ntiles, nx, box, dt );

    // Create cathode
    Cathode cathode( "cathode", +1.0f, make_uint2(4,4), 1.0e5f );

    // Set additional cathode parameters
    cathode.n0 = 1.0f;
    cathode.wall = edge::upper;
    cathode.start = -6.4;
    cathode.uth = make_float3( 0.1, 0.1, 0.1 );

    auto bc = cathode.get_bc();
    bc.x = { 
        .lower = species::bc::open,
        .upper = species::bc::open
    };

    cathode.set_bc( bc );
    sim.add_species( cathode );

    cathode.save_charge();
    cathode.save();
    sim.current -> save( fcomp::x );
    sim.emf -> save( emf::e, fcomp::x );

    float const tmax = 51.2 ;

    while( sim.get_t() < tmax ) {

        sim.advance(); 

        if ( sim.get_iter() % 50 == 0 ) {
            cathode.save_charge();
            cathode.save();
            sim.current -> save( fcomp::x );
            sim.emf -> save( emf::e, fcomp::x );
        }
    }

```



## Initialization

In [1]:
import visxd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

import ipywidgets as widgets

plt.rcParams["figure.figsize"] = [8,6]

## Cathode injection

### Charge Density

In [2]:
xlim = None
ylim = None

@widgets.interact(
    species=['cathode'], iter=(0, 700, 50))
def view(species = 'cathode', iter = 0):
    src = "CHARGE/" + species + "/" + species + "-charge"
    visxd.plot_data(src, iter, xlim = xlim, ylim = ylim  )

interactive(children=(Dropdown(description='species', options=('cathode',), value='cathode'), IntSlider(value=…

### Electric field

In [3]:
@widgets.interact(
    iter=(0, 700, 50)
)
def view(iter = 0):
    src = "EMF/Ex"
    visxd.plot_data(src, iter, xlim = xlim  )

interactive(children=(IntSlider(value=0, description='iter', max=700, step=50), Output()), _dom_classes=('widg…

### Electric current

In [4]:
@widgets.interact(
    iter=(0, 700, 50)
)
def view(iter = 0):
    src = "CURRENT/Jx"
    visxd.plot_data(src, iter, xlim = xlim  )

interactive(children=(IntSlider(value=0, description='iter', max=700, step=50), Output()), _dom_classes=('widg…

### Cathode particles

In [ ]:
visxd.part2D( "PARTICLES/cathode/particles-cathode-000400.zdf", "x", "y", xlim=[100,104], ylim = [250,256])